In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os

In [11]:
IMG_SIZE = 224
BATCH_SIZE = 8

train_dir = 'dataset/train'
val_dir = 'dataset/validation'
test_dir = 'dataset/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2)
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_gen = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_gen = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=1,
    class_mode='binary',
    shuffle=False
)


Found 1799 images belonging to 2 classes.
Found 370 images belonging to 2 classes.
Found 370 images belonging to 2 classes.


In [12]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [13]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)

c:\Users\masil\OneDrive\Desktop\Project\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 233s 1s/step - accuracy: 0.9734 - loss: 0.0764 - val_accuracy: 0.9297 - val_loss: 0.1968
Epoch 2/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 340s 2s/step - accuracy: 0.9899 - loss: 0.0377 - val_accuracy: 0.8973 - val_loss: 0.3972
Epoch 3/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 343s 2s/step - accuracy: 0.9984 - loss: 0.0029 - val_accuracy: 0.9514 - val_loss: 0.1826
Epoch 4/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 1637s 7s/step - accuracy: 0.9952 - loss: 0.0120 - val_accuracy: 0.9757 - val_loss: 0.0730
Epoch 5/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 218s 970ms/step - accuracy: 0.9980 - loss: 0.0029 - val_accuracy: 0.9351 - val_loss: 0.2691


In [14]:
# Get predictions
test_gen.reset()
preds = model.predict(test_gen)
y_pred = np.round(preds).astype(int)
y_true = test_gen.classes

# Metrics
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

370/370 ━━━━━━━━━━━━━━━━━━━━ 31s 77ms/step
Classification Report:
                  precision    recall  f1-score   support

    coffee_bean       1.00      0.85      0.92       185
not_coffee_bean       0.87      1.00      0.93       185

       accuracy                           0.93       370
      macro avg       0.94      0.93      0.93       370
   weighted avg       0.94      0.93      0.93       370

Confusion Matrix:
 [[158  27]
 [  0 185]]


In [15]:
# Get predictions
test_gen.reset()
preds = model.predict(test_gen)
y_pred = np.round(preds).astype(int).flatten()
y_true = test_gen.classes

370/370 ━━━━━━━━━━━━━━━━━━━━ 32s 85ms/step


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

# Print results
print(f"✅ Evaluation Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-Score : {f1:.4f}")

print("\n🧾 Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))

print("🧩 Confusion Matrix:")
print(conf_matrix)

✅ Evaluation Metrics:
Accuracy : 0.9270
Precision: 0.8726
Recall   : 1.0000
F1-Score : 0.9320

🧾 Classification Report:
                 precision    recall  f1-score   support

    coffee_bean       1.00      0.85      0.92       185
not_coffee_bean       0.87      1.00      0.93       185

       accuracy                           0.93       370
      macro avg       0.94      0.93      0.93       370
   weighted avg       0.94      0.93      0.93       370

🧩 Confusion Matrix:
[[158  27]
 [  0 185]]


In [17]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]
    if prediction < 0.5:
        print("Object is present!")
    else:
        print("Object is not present.")

# Example:
print("1", predict_image("bean image/sample.png"))      # Replace with your image path
predict_image("bean image/sample2.png")
predict_image("bean image/sample3.png")
predict_image("bean image/sample4.png")
predict_image("bean image/sample5.png")
predict_image("bean image/sample6.png")
predict_image("bean image/image12.jpg")
predict_image("bean image/image1709.jpg")
predict_image("bean image/libwbg.jpg")
predict_image("bean image/beanonhand.jpg")
predict_image("bean image/PIC.png")
predict_image("bean image/liberica.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Object is not present.
1 None
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Object is present!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Object is not present.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
Object is present!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Object is present!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Object is present!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Object is not present.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Object is not present.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Object is present!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Object is present!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Object is not present.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Object is present!


In [18]:
#model.save('beanVer1.h5')